In [ ]:
# Credits
# The notebook only aims at reproducing results from this DS Project: https://data-flair.training/blogs/python-based-project-image-caption-generator-cnn/

In [1]:
#imports 
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.utils import load_img, img_to_array, pad_sequences
#from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import 
from keras.utils import to_categorical
from keras.layers import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
# small library for seeing the progress of loops.
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

2023-05-04 10:11:22.844187: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 10:11:23.793412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_285/2692966055.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


0it [00:00, ?it/s]

In [2]:

# get all imgs with their captions
def all_img_captions(S3file):
    file = S3file.read()
    file = file.decode()
    captions = file.split('\n')
    descriptions ={}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [ caption ]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions
#Data cleaning- lower casing, removing puntuations and words containing numbers
def cleaning_text(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):
            img_caption.replace("-"," ")
            desc = img_caption.split()
            #converts to lowercase
            desc = [word.lower() for word in desc]
            #remove punctuation from each token
            desc = [word.translate(table) for word in desc]
            #remove hanging 's and a 
            desc = [word for word in desc if(len(word)>1)]
            #remove tokens with numbers in them
            desc = [word for word in desc if(word.isalpha())]
            #convert back to string
            img_caption = ' '.join(desc)
            captions[img][i]= img_caption
    return captions
def text_vocabulary(descriptions):
    # build vocabulary of all unique words
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab
#Join all descriptions in one file 
def join_descriptions(descriptions):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc )
    data = "\n".join(lines)
    return data


In [7]:
# read in data

import os
import s3fs

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

BUCKET = "lvancauwe"
FILE_KEY_S3 = "image_caption_generator/Flickr8k.token.txt"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    descriptions = file_in.read().decode()


In [8]:
BUCKET = "lvancauwe"
FILE_KEY_S3 = "image_caption_generator/Flickr8k.token.txt"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    descriptions = all_img_captions(file_in)

ValueError: Invalid endpoint: https://

In [ ]:
print("Length of descriptions =" ,len(descriptions))
#cleaning the descriptions
clean_descriptions = cleaning_text(descriptions)
#building vocabulary 
vocabulary = text_vocabulary(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))
all_descriptions = join_descriptions(clean_descriptions)

In [31]:
BUCKET_OUT = "lvancauwe"
FILE_KEY_OUT_S3 = "image_caption_generator/descriptions.txt"
#with * gives no error and creates new bucket : FILE_PATH_OUT_S3 = BUCKET_OUT + "/*" + FILE_KEY_OUT_S3
#but also works without * and puts it in the existing bucket
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    file_out.write(all_descriptions)


In [ ]:
# For testing GPU usage 
# def extract_features(directory):
#         model = Xception( include_top=False, pooling='avg' )
#         features = {}
#         for img in tqdm(os.listdir(directory)):
#             filename = directory + "/" + img
#             image = Image.open(filename)
#             image = image.resize((299,299))
#             image = np.expand_dims(image, axis=0)
#             #image = preprocess_input(image)
#             image = image/127.5
#             image = image - 1.0
#             feature = model.predict(image)
#             features[img] = feature
#         return features
# #2048 feature vector
# features = extract_features(dataset_images)
# dump(features, open("features.p","wb"))

In [39]:
#load the data 
def load_photos(file):
    photos = file.split("\n")[:-1]
    return photos
def load_clean_descriptions(all_descriptions, photos): 
    #loading clean_descriptions
    descriptions = {}
    for line in all_descriptions.split("\n"):
        words = line.split()
        if len(words)<1 :
            continue
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = '<start> ' + " ".join(image_caption) + ' <end>'
            descriptions[image].append(desc)
    return descriptions
def load_features(photos):
    #loading all features
    all_features = load(open("features.p","rb"))
    #selecting only needed features
    features = {k:all_features[k] for k in photos}
    return features

In [40]:
# load photos
FILE_KEY_S3 = "image_caption_generator/Flickr_8k.trainImages.txt"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    train_imgs = load_photos(file_in.read().decode())
    
train_descriptions = load_clean_descriptions(all_descriptions, train_imgs)

In [45]:
# load features
import pickle
FILE_KEY_S3 = "image_caption_generator/features.p"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    all_features = pickle.loads(file_in.read())
    
photos = train_imgs
train_features = {k:all_features[k] for k in photos}


In [57]:
#converting dictionary to clean list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc
#creating tokenizer class 
#this will vectorise text corpus
#each integer will represent token in dictionary
from keras.preprocessing.text import Tokenizer
def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer
# give each word an index, and store that into tokenizer.p pickle file
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1

In [51]:
# load tokenized vocab
# FILE_KEY_S3 = "image_caption_generator/tokenizer.p"
# FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

# with fs.open(FILE_PATH_S3, mode="rb") as file_in:
#     tokenizer = pickle.loads(file_in.read())
    

In [52]:
#create input-output sequence pairs from the image description.
#data generator, used by model.fit_generator()
def data_generator(descriptions, features, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items():
            #retrieve photo features
            feature = features[key][0]
            input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, description_list, feature)
            yield [[input_image, input_sequence], output_word]
def create_sequences(tokenizer, max_length, desc_list, feature):
    X1, X2, y = list(), list(), list()
    # walk through each description for the image
    for desc in desc_list:
        # encode the sequence
        seq = tokenizer.texts_to_sequences([desc])[0]
        # split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            # split into input and output pair
            in_seq, out_seq = seq[:i], seq[i]
            # pad input sequence
            in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
            # encode output sequence
            out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
            # store
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)
    return np.array(X1), np.array(X2), np.array(y)


In [58]:
#You can check the shape of the input and output for your model
max_length = 32
[a,b],c = next(data_generator(train_descriptions, train_features, tokenizer, max_length))

In [59]:
#sanity check
print(a.shape, b.shape, c.shape)

(47, 2048) (47, 32) (47, 7577)


In [60]:
from keras.utils import plot_model
# define the captioning model
def define_model(vocab_size, max_length):
    # features from the CNN model squeezed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    # Merging both models
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [ ]:
# train our model
# print('Dataset: ', len(train_imgs))
# print('Descriptions: train=', len(train_descriptions))
# print('Photos: train=', len(train_features))
# print('Vocabulary Size:', vocab_size)
# print('Description Length: ', max_length)
model = define_model(vocab_size, max_length)
epochs = 10
steps = len(train_descriptions)

# making a directory models to save our models
BUCKET_OUT = "lvancauwe"
FILE_KEY_OUT_S3 = "image_caption_generator/models/"
#with * gives no error and creates new bucket : FILE_PATH_OUT_S3 = BUCKET_OUT + "/*" + FILE_KEY_OUT_S3
#but also works without * and puts it in the existing bucket
FILE_PATH_OUT_S3 = BUCKET_OUT + "/*" + FILE_KEY_OUT_S3

for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit_generator(generator, epochs=1, steps_per_epoch= steps, verbose=1)
    with fs.open(FILE_PATH_OUT_S3+ "model_" + str(i) + ".h5", 'w') as file_out:
        file_out.save(model)
